In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.initializers import Constant
from tensorflow.keras import optimizers

In [0]:
from google.colab import drive
drive.mount('/gdrive')

#KERAS

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

## prepare dataset

In [0]:
df = pd.read_csv("sentiment.csv")
df.head()

In [0]:
def get_word2id(df):
  vocab=set()
  for i in df.iteritems():
    words= i[1].split(' ')
    vocab.update(words)
  word2id= dict()
  id2word= dict()
  for i,word in enumerate(vocab):
    word2id[word]=i+1
    id2word[i+1] =word
  return word2id,id2word,vocab

In [0]:
def convert_data(df,word2id):
  new_dataset=[]
  for r in df.iteritems():
    new_dataset.append([word2id[w] for w in r[1].split(" ")])
  return new_dataset

In [0]:
word2id,id2word,vocab = get_word2id(df['sentence'])
new_dataset = convert_data(df['sentence'],word2id)

In [0]:
vocab_size = len(word2id)

In [0]:
new_dataset = pad_sequences(new_dataset, maxlen=128, dtype='int32', padding='post', truncating='post', value=0.0)


In [0]:
new_dataset = np.concatenate([new_dataset,df['polarity'].values.reshape(-1,1)],axis=1)

### split train , test

In [0]:
train = new_dataset[:int(len(new_dataset)*.8)]
test = new_dataset[int(len(new_dataset)*.8):]

### evaluation

In [0]:
def evaluate(y_true,y_pred):
  return confusion_matrix(y_true, y_pred, labels=[0,1])

In [0]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation=tf.nn.relu))
model.add(keras.layers.Dense(16,activation=tf.nn.relu))
model.add(keras.layers.Dense(2,activation=tf.nn.softmax))
model.summary()

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x = train[:,:-1],y=train[:,-1],batch_size=32,epochs=5,validation_split=0.2)

## Sentiment Analysis with LSTM and pre-trained word embeddings (Glove)

### download and prepare embedding

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-12-23 22:42:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-23 22:42:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-23 22:42:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [0]:
!unzip glove.6B.zip.1

Archive:  glove.6B.zip.1
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import csv
import pandas as pd
with open('glove.6B.50d.txt') as f:
    rows, cols = 400000,50

vectorsdf = pd.read_csv(
    'glove.6B.50d.txt', sep=' ',  header=None, index_col=0,
    quoting=csv.QUOTE_NONE, encoding='utf-8')

# remove one junk column
vectorsdf = vectorsdf.dropna(axis=1)
assert vectorsdf.shape == (int(rows), int(cols))
vectorsdf.head()


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
the,0.418000,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.656600,0.278430,-0.147670,-0.55677,0.14658,-0.00951,0.011658,0.102040,-0.127920,-0.84430,-0.121810,-0.016801,-0.332790,-0.155200,-0.231310,-0.191810,-1.8823,-0.76746,0.099051,-0.421250,-0.19526,4.0071,-0.185940,-0.522870,-0.31681,0.000592,0.007445,0.17778,-0.158970,0.012041,-0.054223,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
",",0.013441,0.236820,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.239380,0.130010,-0.063734,-0.39575,-0.48162,0.23291,0.090201,-0.133240,0.078639,-0.41634,-0.154280,0.100680,0.488910,0.312260,-0.125200,-0.037512,-1.5179,0.12612,-0.024420,-0.042961,-0.28351,3.5416,-0.119560,-0.014533,-0.14990,0.218640,-0.334120,-0.13872,0.318060,0.703580,0.448580,-0.080262,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.378180,-0.566570,0.044691,0.30392
.,0.151640,0.301770,-0.16763,0.17684,0.31719,0.339730,-0.43478,-0.31086,-0.44999,-0.294860,0.166080,0.119630,-0.41328,-0.42353,0.59868,0.288250,-0.115470,-0.041848,-0.67989,-0.250630,0.184720,0.086876,0.465820,0.015035,0.043474,-1.4671,-0.30384,-0.023441,0.305890,-0.21785,3.7460,0.004228,-0.184360,-0.46209,0.098329,-0.119070,0.23919,0.116100,0.417050,0.056763,-0.000064,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.080803,-0.356520,0.016413,0.10216
of,0.708530,0.570880,-0.47160,0.18048,0.54449,0.726030,0.18157,-0.52393,0.10381,-0.175660,0.078852,-0.362160,-0.11829,-0.83336,0.11917,-0.166050,0.061555,-0.012719,-0.56623,0.013616,0.228510,-0.143960,-0.067549,-0.381570,-0.236980,-1.7037,-0.86692,-0.267040,-0.258900,0.17670,3.8676,-0.161300,-0.132730,-0.68881,0.184440,0.005246,-0.33874,-0.078956,0.241850,0.365760,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.80375
to,0.680470,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,-0.085253,0.171180,0.224190,-0.10046,-0.43653,0.33418,0.678460,0.057204,-0.344480,-0.42785,-0.432750,0.559630,0.100320,0.186770,-0.268540,0.037334,-2.0932,0.22171,-0.398680,0.209120,-0.55725,3.8826,0.474660,-0.956580,-0.37788,0.208690,-0.327520,0.12751,0.088359,0.163510,-0.216340,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


### remove words which are not in embeddings


In [0]:
cvocab =vocab.copy()
for v in cvocab:
  if v not in vectorsdf.index:
    vocab.remove(v)

### handle UNKOWN words 

In [0]:
word_to_ids={}
vocab_vectors=[]
e_dim = 50

# for i,v in enumerate(vocab):
#   vocab_vectors.append(vectorsdf.loc[v].values.tolist())
#   word_to_ids[v]=i
for i,v in enumerate(vectorsdf.iterrows()):
  word_to_ids[v[0]]=i

vocab_vectors=vectorsdf.values

vocab_vectors=np.array(vocab_vectors)
word_to_ids["<UNK>"]=len(vocab)
word_to_ids["<PAD>"]=0
vocab_vectors = np.concatenate([vocab_vectors,np.ones((1,e_dim))],axis=0)
vocab_vectors=np.concatenate([vocab_vectors,np.ones((1,e_dim))*20],axis=0)

### prepare data

In [0]:
def convert_data(df,word2id):
  new_dataset=[]
  for r in df.iteritems():
      s=[]
      for w in r[1].split(" "):
        if w in word2id:
          s.append(word2id[w])
        else:
          s.append(word2id['<UNK>'])
      new_dataset.append(s)    
  return new_dataset

In [0]:
new_dataset = convert_data(df['sentence'],word_to_ids)
new_dataset = pad_sequences(new_dataset, maxlen=128, dtype='int32', padding='post', truncating='post', value=0.0)
new_dataset = np.concatenate([new_dataset,df['polarity'].values.reshape(-1,1)],axis=1)

In [0]:
train = new_dataset[:int(len(new_dataset)*.8)]
test = new_dataset[int(len(new_dataset)*.8):]

### build model

In [0]:
from tensorflow import keras
model = keras.Sequential()

model.add(keras.layers.Embedding(vocab_vectors.shape[0],
                            50,
                            embeddings_initializer=Constant(vocab_vectors),
                            input_length=128,
                            trainable=True))

model.add(keras.layers.LSTM(32,activation=tf.nn.tanh))
model.add(keras.layers.Dense(2,activation=tf.nn.softmax))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 128, 50)           20000100  
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 66        
Total params: 20,010,790
Trainable params: 20,010,790
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x = train[:,:-1],y=train[:,-1],batch_size=4000,epochs=30,validation_split=0.2)

Train on 32000 samples, validate on 8000 samples
Epoch 1/30
32000/32000 [==============================] - 4s 124us/sample - loss: 0.6930 - acc: 0.5131 - val_loss: 0.6926 - val_acc: 0.5170
Epoch 2/30
32000/32000 [==============================] - 2s 68us/sample - loss: 0.6892 - acc: 0.5363 - val_loss: 0.6896 - val_acc: 0.5397
Epoch 3/30
32000/32000 [==============================] - 2s 67us/sample - loss: 0.6857 - acc: 0.5557 - val_loss: 0.6867 - val_acc: 0.5530
Epoch 4/30
32000/32000 [==============================] - 2s 68us/sample - loss: 0.6813 - acc: 0.5677 - val_loss: 0.6818 - val_acc: 0.5604
Epoch 5/30
32000/32000 [==============================] - 2s 67us/sample - loss: 0.6725 - acc: 0.5877 - val_loss: 0.6697 - val_acc: 0.5888
Epoch 6/30
32000/32000 [==============================] - 2s 67us/sample - loss: 0.6366 - acc: 0.6427 - val_loss: 0.6280 - val_acc: 0.6795
Epoch 7/30
32000/32000 [==============================] - 2s 67us/sample - loss: 0.5990 - acc: 0.6956 - val_loss: 0.

In [0]:
predicts = model.predict(train[:,:-1])
evaluate(train[:,-1],predicts)

array([51475,  1645, 39211, 51475, 51475, 51021, 26773,  7784, 40901,
       33471, 46286, 24964, 51475, 51475, 11268, 51475, 11268, 51475,
       51475, 51475, 11268, 33514, 47844, 39002, 27407, 33514, 43300,
       42118, 17699, 24565, 43877, 46395, 12199, 51475, 49453, 51475,
       51475, 26608, 45030, 32702, 50970, 36536, 26562,  5672, 23960,
       31028, 41316, 45137, 20395, 11268, 45137,  2726, 51475, 51475,
       51475, 45137, 51037, 51417, 51475, 11268, 51475,  1519,  4962,
       40392, 40555, 50970, 51475, 51475, 51475, 23740,  7784, 10113,
       43877, 11893, 23170,  6598, 51475, 51475, 51475, 50970, 51475,
        7784, 28580, 23170, 17699, 51475, 51475, 51475,  8521, 43877,
       17699, 10682, 42658, 23170, 17699, 51475, 51475, 51475, 51475,
       51475, 51475, 51475,  8830, 51475, 21227, 39901, 12329,  8296,
       42656, 50970, 36536, 51475, 51475, 51475, 45137, 43454, 51475,
       51475, 51475, 51475,  1927, 51058, 17383, 11582, 28058, 51475,
       51475, 51475]